0. Phì đại động mạch chủ
1. Xẹp phổi
2. Sự vôi hóa
3. Tim to
4. Củng cố
5. ILD
6. Sự xâm nhập
7. Độ mờ của phổi
8. Nodule / Mass
9. Tổn thương khác
10. Tràn dịch màng phổi
11. Dày màng phổi
12. Tràn khí màng phổi
13. Xơ phổi
14. "Không tìm thấy"

In [ ]:
!pip install bbox_visualizer

## Thêm thư viện

In [ ]:
import numpy as np
import pandas as pd 
import os
from sklearn.model_selection import train_test_split
from sklearn import model_selection

import cv2
import matplotlib.pyplot as plt
import seaborn as sns
import bbox_visualizer as bbv

import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
from glob import glob
from skimage import exposure

import torch

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SequentialSampler
import torchvision

from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

import warnings

warnings.filterwarnings('ignore')

In [ ]:
BASE_DIR = '../input/vinbigdata-chest-xray-abnormalities-detection/'
WEIGHTS_FILE = './'

In [ ]:
dataset = pd.read_csv(os.path.join(BASE_DIR, "train.csv")) # đưa dữ liệu vào pandas
dataset.head()

In [ ]:
# vẽ biểu đồ
plt.figure(figsize=(16, 9))

sns.countplot(dataset.class_name)
plt.xticks(fontsize=14,rotation=65)
plt.yticks(fontsize=14)
plt.title("Phân phối nhãn trong khung dữ liệu chú thích", fontsize=16)

In [ ]:
dataset_new = dataset[dataset.class_name!='No finding'].reset_index(drop=True) # bỏ đi nhãn no finding và reset lại vị trí

# vẽ biểu đồ sau khi loại bỏ no finding
plt.figure(figsize=(16, 9))

sns.countplot(dataset_new.class_name)
plt.xticks(fontsize=14,rotation=65)
plt.yticks(fontsize=14)
plt.title("Phân phối nhãn trong khung dữ liệu chú thích", fontsize=16);

In [ ]:
train_df, valid_df = train_test_split(dataset_new, test_size=0.33, random_state=42) # chia dữ liệu

## Thiết kế Dataset Loader

In [ ]:
# Xây dựng Tập dữ liệu chú thích phổi
class LungsAnnotationDataset(Dataset):
    def __init__(self,dataframe, image_dir, transforms=None):
        super().__init__()
        
        self.image_ids = dataframe['image_id'].unique()
        self.df = dataframe
        self.image_dir = image_dir
        self.transforms = transforms
    
    def __getitem__(self, index: int):
        image_id = self.image_ids[index]
        records = self.df[self.df['image_id'] == image_id]
        # đọc file dicom
        dcm_data = pydicom.read_file(f'{self.image_dir}/{image_id}.dicom')
        image = apply_voi_lut(dcm_data.pixel_array, dcm_data)
        # tùy thuộc vào giá trị này, tia X có thể bị đảo ngược
        if dcm_data.PhotometricInterpretation == "MONOCHROME1":
            image = np.amax(image) - image
            
        image = np.stack([image, image, image])
        image = image - np.min(image)

        image = image / image.max()

        image = exposure.equalize_hist(image) # Chuẩn hóa hình ảnh tia X
        image = image.astype('float32')

        image = image.transpose(1,2,0)
        
        boxes = records[['x_min','y_min','x_max','y_max']].values #boxes[:, 2] = boxes[:, 0] + boxes[:, 2]
        
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        area = torch.as_tensor(area, dtype=torch.float32)
        
        labels = records.class_id.values + 1
        class_name = records.class_name.values
        #giả sử tất cả các trường hợp không phải là đa số
        iscrowd = torch.zeros((records.shape[0],), dtype=torch.int64)
        
        target = {}
        target['boxes'] = torch.tensor(boxes)
        target['labels'] = torch.tensor(labels)
        target['area'] = torch.tensor(area)
        target['iscrowd'] = torch.tensor(iscrowd)
        
        if self.transforms:
            sample = {
                'image': image,
                'bboxes': target['boxes'],
                'labels': labels
            }
            sample = self.transforms(**sample)
            image = sample['image']
            
            target['boxes'] = torch.tensor(sample['bboxes'])
            
        return image, target
    
    def __len__(self) -> int:
        return self.image_ids.shape[0]

In [ ]:
# Albumentations
# ToTensorV2: Chuyển đổi hình ảnh và mặt nạ thành` torch.Tensor`. 
# Hình ảnh `HWC` numpy được chuyển đổi thành pytorch` CHW` tensor.
def get_train_transform():
    return A.Compose([ToTensorV2(),], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})

def get_valid_transform():
    return A.Compose([ToTensorV2(),], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})

In [ ]:
DIR_TRAIN = os.path.join(BASE_DIR, "train")
# gắn nhãn cho bộ dữ liệu
train_dataset = LungsAnnotationDataset(train_df, DIR_TRAIN,get_train_transform())
valid_dataset = LungsAnnotationDataset(valid_df, DIR_TRAIN,get_valid_transform())

In [ ]:
def collate_fn(batch):
    return tuple(zip(*batch))

train_data_loader = DataLoader(
    train_dataset,
    batch_size=6,
    shuffle=False,
    num_workers=4,
    collate_fn=collate_fn
) # xử lý 1 loạt 6 ảnh và 6 nhãn liên tiếp theo batch_size

valid_data_loader = DataLoader(
    valid_dataset,
    batch_size=6,
    shuffle=False,
    num_workers=4,
    collate_fn=collate_fn
) # xử lý 1 loạt 6 ảnh và 6 nhãn liên tiếp theo batch_size

In [ ]:
# device = torch.device('cuda') 
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [ ]:
#iter() gọi __iter__()phương thức iris_loader trả về một trình lặp. 
#next() sau đó gọi __next__()phương thức trên trình lặp đó để nhận lần lặp đầu tiên. 
#Chạy next()lại sẽ nhận được mục thứ hai của trình lặp, v.v
images, targets = next(iter(train_data_loader))
images = list(image.to(device) for image in images)

targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

In [ ]:
class_brands = {
    0: 'Aortic enlargement',
    1: 'Atelectasis',
    2: 'Calcification',
    3: 'Cardiomegaly',
    4: 'Consolidation',
    5: 'ILD',
    6: 'Infiltration',
    7: 'Lung Opacity',
    8: 'Nodule/Mass',
    9: 'Other lesion',
    10: 'Pleural effusion',
    11: 'Pleural thickening',
    12: 'Pneumothorax',
    13: 'Pulmonary fibrosis'
}

## Trực quan hóa các bất thường trên X-quang ngực

In [ ]:
def plot_x_ray(idx,images,targets):
    boxes = targets[idx]['boxes'].cpu().numpy().astype(np.int32)
    labels = targets[idx]['labels']-1
    sample = images[idx].permute(1,2,0).cpu().numpy()
    
    img = sample.copy()
    plt.figure(figsize=(16, 16))
    for box,label in zip(boxes,labels):
        bbv.add_label(img, class_brands[label.item()], box, 
                      draw_bg=True,
                      text_bg_color=(255,0,0),
                      text_color=(0,0,0),
                      )
        cv2.rectangle(img ,
                      (box[0], box[1]),
                      (box[2], box[3]),
                      (255,0,0), 3)

    plt.imshow(img)    

In [ ]:
plot_x_ray(2,images,targets) # hiển thị các loại bất thường kèm vị trí

In [ ]:
plot_x_ray(3,images,targets)

In [ ]:
plot_x_ray(1,images,targets)

## Xây dựng Mô hình

In [ ]:
# Thay thế classifier bằng một trình phân loại mới, có num_classes do người dùng xác định
num_classes = 15  # 14 lớp + background

# tải một mô hình; được đào tạo trước về COCO
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
# nhận số lượng các đặc trưng đầu vào cho classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features

# thay thế đầu được đào tạo trước bằng một đầu mới của FastRcnn
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

In [ ]:
model.to(device)
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.0005, momentum=0.9, weight_decay=0.0005) #khởi tạo optimizer.
lr_scheduler = None

# chọn sô epoch = 4
num_epochs = 4

In [ ]:
# hàm tính loss
class Averager:
    def __init__(self):
        self.current_total = 0.0
        self.iterations = 0.0

    def send(self, value):
        self.current_total += value
        self.iterations += 1

    @property
    def value(self):
        if self.iterations == 0:
            return 0
        else:
            return 1.0 * self.current_total / self.iterations

    def reset(self):
        self.current_total = 0.0
        self.iterations = 0.0

In [ ]:
def plot_validation(idx, images, targets, predicted):
    boxes = targets[idx]['boxes'].cpu().numpy().astype(np.int32)
    labels = targets[idx]['labels']-1
    
    boxes_pred = predicted[idx]['boxes']#.detach().numpy().astype(np.int32)
    labels_pred = predicted[idx]['labels']-1
    scores = predicted[idx]['scores'].data.cpu().numpy()
    # Threshold: ngưỡng chấp nhận
    boxes_pred = boxes_pred[scores >= 0.2]
    
    sample = images[idx].permute(1,2,0).cpu().numpy()
    
    img = sample.copy() # tạo 1 mẫu bản sao
    plt.figure(figsize=(16, 16))
    
    for box,label in zip(boxes,labels):
        bbv.add_label(img, class_brands[label.item()], box, 
                      draw_bg=True,
                      text_bg_color=(255,0,0),
                      text_color=(0,0,0),
                      )
        cv2.rectangle(img ,
                      (box[0], box[1]),
                      (box[2], box[3]),
                      (255,0,0), 3)
    for box,label in zip(boxes_pred,labels_pred):
        bbv.add_label(img, class_brands[label.item()], box, 
                      draw_bg=True,
                      text_bg_color=(255, 255, 0), #đỏ
                      text_color=(0,0,0),
                      )
        cv2.rectangle(img ,
                      (box[0], box[1]),
                      (box[2], box[3]),
                      (255, 255, 0), 3) # vàng

    plt.imshow(img)
    plt.title('Vùng tin tưởng - Đỏ, Hộp đã dự đoán - Vàng',fontsize=14)

In [ ]:
def train_model(train_dataset, fold):
    train_data_loader = DataLoader(
        train_dataset,
        batch_size=2,
        shuffle=False,
        num_workers=4,
        collate_fn=collate_fn
    )
    loss_hist = Averager() # khởi tạo 1 đối tượng tính trung bình
    itr = 1
    for epoch in range(num_epochs):
        loss_hist.reset() #lấy lại giá trị gốc
        model.train()
        for images, targets in train_data_loader:
            optimizer.zero_grad()
            images = list(image.to(device) for image in images)
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

            loss_dict = model(images, targets)

            loss_classifier, loss_box_reg, loss_objectness, loss_rpn_box_reg = loss_dict.values()
            losses = sum([loss_objectness, 
                         10*loss_classifier, 
                         10*loss_rpn_box_reg, 
                         (0.5*loss_box_reg**2)
                         ]
                        )
            loss_value = losses.item()
            loss_hist.send(loss_value)

            losses.backward()
            optimizer.step()
            if itr%100==0:
                print(f"Fold #{fold} Epoch #{epoch+1} Iteration #{itr} loss: {loss_hist.value}")
            
            itr += 1
            del loss_dict, loss_classifier, loss_box_reg, loss_objectness, loss_rpn_box_reg,loss_value
        itr=1    
        # cập nhật tốc độ học
        if lr_scheduler is not None:
            lr_scheduler.step()

        print(f"Fold #{fold} Epoch #{epoch+1} loss: {loss_hist.value}")
        print("Saving Epoch's state...")
        torch.save(model.state_dict(), f"model_fasterRCNN.pth") #lưu model để tái sử dụng

## Huấn luyện cho mô hình

In [ ]:
k = 1
df = dataset_new.sample(frac=1).reset_index(drop=True)
y = dataset_new.class_id.values
## Chia tỉ lệ bằng K-fold
kfold = model_selection.GroupKFold(n_splits=5)

for train_index, val_index in kfold.split(df, y,groups=df.image_id.values):
    train_dataset = LungsAnnotationDataset(df.loc[val_index], DIR_TRAIN,get_train_transform())
    train_model(train_dataset, k)
    if k==5:
        valid_dataset = LungsAnnotationDataset(df.loc[train_index], DIR_TRAIN,get_train_transform())
        val_data_loader = DataLoader(valid_dataset,
                                batch_size=6,
                                shuffle=False,
                                num_workers=4,
                                collate_fn=collate_fn
                            ) # tải dữ liệu valid vs bó là 6
        
    k += 1

In [ ]:
iter_ = iter(val_data_loader) #lặp
images_val0, targets_val0 = next(iter_) # nhận lần lặp đầu tiên
images_val1, targets_val1 = next(iter_) # nhận lần lặp thứ 2

## Load lại mô hình

In [ ]:
model.load_state_dict(torch.load(WEIGHTS_FILE+'model_fasterRCNN.pth', map_location=device)) # load lại model đã save ở trên
model.eval() #tắt các lớp như Dropouts Layers, BatchNorm Layers, v.v trong khi đánh giá mô hình

model.to(device) # chỉ cần chuyển đổi mô hình đã khởi tạo thành cpu or cuda tùy thuộc tham số trên
model.eval()
device = torch.device("cuda")
images_0 = list(image.to(device) for image in images_val0)
outputs0 = model(images_0)
outputs0 = [{k: v.to(device) for k, v in t.items()} for t in outputs0]

In [ ]:
images_1 = list(image_.to(device) for image_ in images_val1)
outputs1 = model(images_1)
outputs1 = [{k: v.to(device) for k, v in t.items()} for t in outputs1]

## Trực quan hóa hiệu suất của mô hình qua hình ảnh validation

In [ ]:
plot_validation(0, images_val0, targets_val0, outputs0)

In [ ]:
plot_validation(0, images_val1, targets_val1, outputs1)

In [ ]:
plot_validation(1, images_val0, targets_val0, outputs0)

In [ ]:
plot_validation(1, images_val1, targets_val1, outputs1)

In [ ]:
plot_validation(2, images_val0, targets_val0, outputs0)

In [ ]:
plot_validation(2, images_val1, targets_val1, outputs1)

In [ ]:
plot_validation(3, images_val0, targets_val0, outputs0)

In [ ]:
plot_validation(3, images_val1, targets_val1, outputs1)

In [ ]:
plot_validation(4, images_val0, targets_val0, outputs0)

In [ ]:
plot_validation(4, images_val1, targets_val1, outputs1)

In [ ]:
plot_validation(5, images_val0, targets_val0, outputs0)

In [ ]:
plot_validation(5, images_val1, targets_val1, outputs1)